<a href="https://colab.research.google.com/github/Crawnicles/Algo-trading-project/blob/main/VolatilityBreakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project setup
Configure imports and setup the connection to the Alpaca API

In [35]:
!pip install python_dotenv
!pip install alpaca_trade_api
!pip install backtrader

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [60]:
from alpaca_trade_api.rest import TimeFrame
import alpaca_trade_api as tradeapi
from alpaca_trade_api.stream import Stream
import backtrader as bt
import matplotlib
import pandas as pd
from alpaca_trade_api import TimeFrameUnit
from pandas.core.frame import DataFrame
import numpy as np

In [57]:
# Alpaca API info
ALPACA_API_KEY = 'PKZ7UQDMAWO5UAULDXCO'
ALPACA_SECRET_KEY = 'ESBhzVmXVPtUjbzdmKx3airlPQ5eBHlbamnNy123'

rest_api = tradeapi.REST(ALPACA_API_KEY,
                ALPACA_SECRET_KEY, 
                'https://paper-api.alpaca.markets',
                'v2')

# Configure Base algorithim

# Gather Data Frames

In [92]:
spy_bars = rest_api.get_bars('SPY', TimeFrame.Day, '2020-01-01', '2020-12-31').df
spy_bars

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2020-01-02 05:00:00+00:00,296.27,300.6000,295.1900,300.35,33920004,283490,298.439935
2020-01-03 05:00:00+00:00,296.95,300.5800,296.5000,297.43,36633995,280942,298.810386
2020-01-06 05:00:00+00:00,293.70,299.9600,292.7500,299.80,29644779,242867,297.799601
2020-01-07 05:00:00+00:00,299.98,300.9000,297.4800,298.39,27885664,227873,298.924596
2020-01-08 05:00:00+00:00,297.17,304.4399,297.1560,303.19,33091741,261777,301.165533
...,...,...,...,...,...,...,...
2020-12-24 05:00:00+00:00,131.41,133.4600,131.1000,131.97,54930664,365719,132.333929
2020-12-28 05:00:00+00:00,134.00,137.3400,133.5100,136.69,124490748,782413,135.841386
2020-12-29 05:00:00+00:00,138.05,138.7890,134.3409,134.87,121048953,797017,136.376624


In [86]:
def VolBreakout(PD_Dataframe, percentage: int):
  # Convert int to Percentage
  multiplier = percentage / 100
  
  # Calculate the Top and Bottom of the range to determine where positions will be taken
  top_range_calc = PD_Dataframe['high'] * multiplier
  bottom_range_calc = PD_Dataframe['low'] * multiplier
  top_of_range = PD_Dataframe['open'] + top_range_calc
  bottom_of_range = PD_Dataframe['open'] - bottom_range_calc

  # Initiate a Column that counts the number of times a cross happens on above the range and below the range
  PD_Dataframe['Crossover Range'] = 0.0
  PD_Dataframe['Crossunder Range'] = 0.0

  # Find the columns where a cross happens and updates the column with a 1 if there is a cross
  PD_Dataframe['Crossover Range'] = np.where(PD_Dataframe['close'] > top_of_range, 1.0, 0.0)
  PD_Dataframe['Crossunder Range'] = np.where(PD_Dataframe['close'] < bottom_of_range, 1.0, 0.0)

  # Creates two columns to indicate a Long or Short Signal
  PD_Dataframe['Long Signal'] = PD_Dataframe['Crossover Range'].diff()
  PD_Dataframe['Short Signal'] = PD_Dataframe['Crossunder Range'].diff()

  return(PD_Dataframe)

In [93]:
VolBreakout(spy_bars, 2)

,open,high,low,close,volume,trade_count,vwap,Crossover Range,Crossunder Range,Long Signal,Short Signal
timestamp,,,,,,,,,,,
2020-01-02 05:00:00+00:00,296.27,300.6000,295.1900,300.35,33920004,283490,298.439935,0.0,0.0,NaN,NaN
2020-01-03 05:00:00+00:00,296.95,300.5800,296.5000,297.43,36633995,280942,298.810386,0.0,0.0,0.0,0.0
2020-01-06 05:00:00+00:00,293.70,299.9600,292.7500,299.80,29644779,242867,297.799601,1.0,0.0,1.0,0.0
2020-01-07 05:00:00+00:00,299.98,300.9000,297.4800,298.39,27885664,227873,298.924596,0.0,0.0,-1.0,0.0
2020-01-08 05:00:00+00:00,297.17,304.4399,297.1560,303.19,33091741,261777,301.165533,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24 05:00:00+00:00,131.41,133.4600,131.1000,131.97,54930664,365719,132.333929,0.0,0.0,0.0,0.0
2020-12-28 05:00:00+00:00,134.00,137.3400,133.5100,136.69,124490748,782413,135.841386,0.0,0.0,0.0,0.0
2020-12-29 05:00:00+00:00,138.05,138.7890,134.3409,134.87,121048953,797017,136.376624,0.0,1.0,0.0,1.0


In [39]:
# Creates a Function to perform backtesting using the alpaca API
def run_backtest(strategy, symbols, start, end, timeframe=TimeFrame.Day, cash=10000):
	"""params:
          strategy: the strategy you wish to backtest, an instance of backtrader.Strategy
          symbols: the symbol (str) or list of symbols List[str] you wish to backtest on
          start: start date of backtest in format 'YYYY-MM-DD'
          end: end date of backtest in format: 'YYYY-MM-DD'
          timeframe: the timeframe the strategy trades on (size of bars) -
                      1 min: TimeFrame.Minute, 1 day: TimeFrame.Day, 5 min: TimeFrame(5, TimeFrameUnit.Minute)
          cash: the starting cash of backtest
    """

	# initialize backtrader broker
	cerebro = bt.Cerebro(stdstats=True)
	cerebro.broker.setcash(cash)

	# add strategy
	cerebro.addstrategy(strategy)

	# add analytics
	# cerebro.addobserver(bt.observers.Value)
	# cerebro.addobserver(bt.observers.BuySell)
	cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
	# historical data request
	if type(symbols) == str:
		symbol = symbols
		alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
		data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
		cerebro.adddata(data)
	elif type(symbols) == list or type(symbols) == set:
		for symbol in symbols:
			alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
			data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
			cerebro.adddata(data)


	# run
	initial_portfolio_value = cerebro.broker.getvalue()
	print(f'Starting Portfolio Value: {initial_portfolio_value}')
	results = cerebro.run()
	final_portfolio_value = cerebro.broker.getvalue()
	print(f'Final Portfolio Value: {final_portfolio_value} ---> Return: {(final_portfolio_value/initial_portfolio_value - 1)*100}%')

	strat = results[0]
	print('Sharpe Ratio:', strat.analyzers.mysharpe.get_analysis()['sharperatio'])
	cerebro.plot(iplot= False)